In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

from tfx.proto import example_gen_pb2
from tfx.v1.proto import TrainArgs, EvalArgs, Input, Output, PushDestination
from tfx.components import ImportExampleGen, StatisticsGen, SchemaGen, Transform, Trainer, Evaluator, Pusher
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

In [2]:
_pipeline_name = 'adult_keras'
_adult_root = os.path.join(os.environ['HOME'], 'adult')
_data_root = os.path.join(_adult_root, 'data')
_serving_model_dir = os.path.join(_adult_root, 'serving_model', _pipeline_name)   ## path to save the trained model
_tfx_root = os.path.join(os.environ['HOME'], 'tfx')
absl.logging.set_verbosity(absl.logging.INFO)

## noise_type = 0 : add noise on the input data
## noise_type = 1: add noise on gradients
noise_type = 1

In [3]:
context = InteractiveContext()

In [4]:
input_cfg = Input(splits=[example_gen_pb2.Input.Split(name='train', pattern='train/*'),
                          example_gen_pb2.Input.Split(name='eval', pattern='eval/*')])

example_gen = ImportExampleGen(input_base=_data_root, input_config=input_cfg)
context.run(example_gen)

INFO:absl:Running driver for ImportExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for ImportExampleGen
INFO:absl:Generating examples.


INFO:absl:Reading input TFRecord data /home/liweiting/adult/data/train/*.
INFO:absl:Reading input TFRecord data /home/liweiting/adult/data/eval/*.
INFO:absl:Examples generated.
INFO:absl:Running publisher for ImportExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/tmp/tfx-interactive-2021-11-30T09_56_16.483723-30vi6r_a/ImportExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:train,num_files:1,total_bytes:1381840,xor_checksum:1637219293,sum_checksum:1637219293\nsplit:eval,num_files:1,total_bytes:296664,xor_checksum:1637219275,sum_checksum:1637219275"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.2"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [5]:
import tensorflow as tf 
import numpy as np 
from tfx.dsl.component.experimental.decorators import component
from tfx.v1.dsl.components import InputArtifact, OutputArtifact, Parameter, OutputDict
from tfx.types.standard_artifacts import Examples, Float, Model, String
from adult_model import NN

feature_num = 14

@component
def Estimator(data:InputArtifact[Examples],
              significance: OutputArtifact[String],
              model_dir: Parameter[str]):
    feature_description = {}
    feature_description['feature'] = tf.io.VarLenFeature(tf.float32)
    feature_description['label'] = tf.io.VarLenFeature(tf.float32)
    feature_description['block'] = tf.io.VarLenFeature(tf.int64)
    
    tags = ['feature', 'label', 'block']
    
    def read_and_decode(example_string):
        feature_dict = tf.io.parse_single_example(example_string, feature_description)
        outputs = [feature_dict[tags[i]] for i in range(len(tags))]
        return outputs
    
    train_uri = os.path.join(data.uri, 'Split-train')
    train_filenames = [os.path.join(train_uri, name) for name in os.listdir(train_uri)]
    trainset = tf.data.TFRecordDataset(train_filenames, compression_type='GZIP')
    blocks = trainset.map(read_and_decode)
    
    val_uri = os.path.join(data.uri, 'Split-eval')
    val_filenames = [os.path.join(val_uri, name) for name in os.listdir(val_uri)]
    valset = tf.data.TFRecordDataset(val_filenames, compression_type='GZIP')
    val_data = valset.map(read_and_decode) 
    
    # Only one item in val_data set and here aiming to extract this item
    for val in val_data:
        val_feature, val_label, _ = val
        val_feature = tf.reshape(tf.sparse.to_dense(val_feature), [-1, feature_num])
        val_label = tf.reshape(tf.sparse.to_dense(val_label), [-1,])
    
    def compile_model(model):
        loss_custom = tf.keras.losses.BinaryCrossentropy()
#     lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(0.01, 215, 0.99, staircase=True)
        optimizer = tf.keras.optimizers.Adam()
        model.compile(optimizer=optimizer, loss=loss_custom, metrics=[], run_eagerly=False)
        
        return model
    
    model = NN()
    inputs = tf.random.uniform((1, 14))
    outputs = model(inputs)
    model.load_weights(model_dir)
    model = compile_model(model)
    
    loss_before = model.evaluate(x=val_feature, y=val_label, batch_size=256)
    significance_list = []
    
    for block in blocks:    
        # Deal with the data
        train_feature, train_label, block_idx = block
        train_feature = tf.reshape(tf.sparse.to_dense(train_feature), [-1, feature_num])
        train_label = tf.reshape(tf.sparse.to_dense(train_label), [-1,])
        # Load the current model
        model.load_weights(model_dir)
        model = compile_model(model)
        
        hist = model.fit(x=train_feature, y=train_label, batch_size=250, epochs=10, verbose=0,
                         validation_data=(val_feature, val_label), validation_batch_size=256)
        loss_after = hist.history['val_loss'][-1]
        
        delta_loss = loss_before - loss_after
        significance_list.append(delta_loss)
        
    def normalization(data):
        _range = np.max(data) - np.min(data)
        norm = (data - np.min(data)) / _range
        return norm
    
    # Normalize the significance to [0, 1]
    significance_list = normalization(significance_list)
    print("Sig_list: ", significance_list)
    
    with tf.io.gfile.GFile(significance.uri, 'w') as f:
        for sig in significance_list:
            f.write(str(sig))
            f.write('\n')
    
    return

In [ ]:
estimator = Estimator(data=example_gen.outputs['examples'],
                      model_dir='/home/liweiting/adult/saved_model/weights.h5')
context.run(estimator)

INFO:absl:Running driver for Estimator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Estimator


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity t

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function wi

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function wi

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function wi

In [110]:
uri = estimator.outputs['significance'].get()[0].uri   
sig_list = []
for line in tf.io.gfile.GFile(uri):
    sig_list.append(str(line[:-1]))
print("Sig_list: ", sig_list)
arr = np.array(sig_list)
print(arr.argsort()[-13:][::-1])

Sig_list:  ['0.9457006057296893', '0.9124039602636461', '0.9781412572373276', '0.9391808694973898', '0.9338605378174585', '0.9096480649218235', '0.8984421416462334', '0.9551621247492166', '0.9375813257573647', '0.9674762816230253', '0.9669733218593', '1.0', '0.9868683435706497', '0.9935856168702734', '0.8610225430314241', '0.9413390885842315', '0.9556549544069254', '0.9549281193002326', '0.9141802743536612', '0.9281385474339682', '0.0', '0.9129459209355355', '0.855800979885155']
[11 13 12  2  9 10 16  7 17  0 15  3  8]


In [17]:
_schedule_module_file = "schedule_utils.py"

In [18]:
%%writefile {_schedule_module_file}
import numpy as np 

def threshold_type12(z, U, L):
    threshold = np.power((U*np.e/L), z)*L/np.e
    return threshold

def threshold_type3(z):
    threshold = 0.005 * np.power(z, 2)
    return threshold

def filter_type1(z, U, L, eps, eps0, sig):
    z0 = 1 / (1+np.log(U/L))
    if z<=z0:
        return True
    else:
        noise1 = np.random.laplace(0, 4/eps0, 1)
        noise2 = np.random.laplace(0, 2/eps0, 1)
        if sig/(eps+eps0) + noise1 > threshold_type12(z, U, L) + noise2:
            return True
    
    return False

def filter_type2(z, U, L, sig, gamma, lambd, sens):
    z0 = 1 / (1+np.log(U/L))
    if z<=z0:
        return True
    else:
        noise1 = np.random.laplace(0, 4/eps0, 1)
        noise2 = np.random.laplace(0, 2/eps0, 1)
        if sig*gamma*np.sqrt(1-lambd)/sens + noise1 > threshold_type12(z, U, L) + noise2:
            return True
    
    return False

_U = 1
_L = 0
_eps0 = 1
_beta = 1.5

def schedule(schedule_type, arguments):
    initial_budget, consumed_budget, lambd, gamma, sensitivity, significance = arguments
    # Sage
    if schedule_type == 0:
        epsilon = - sensitivity * np.log(1-lambd) / gamma
    # Infocom
    elif schedule_type == 1:
        epsilon = - sensitivity * np.log(1-lambd) / gamma
        z = consumed_budget / initial_budget
        if filter_type1(z, _U, _L, epsilon, _eps0, significance):
            epsilon = epsilon + eps0
        else:
            epsilon = 0
    # RPBS     
    elif schedule_type == 2:
        b = (gamma / sensitivity) * [1/lambd -1 + np.sqrt((1/lambd)*(1/lambd -1))]
        epsilon = np.random.gamma(2, 1/b, 1)
        if not filter_type2(z, U, L, significance, gamma, lambd, sensitivity):
            epsilon = 0

    # Extended RPBS
    elif schedule_type == 3:
        c = gamma / sensitivity
        z = consumed_budget / initial_budget
        T = threshold_type3(z)
        delta = (T-c)*(T-c) - 4*(_beta-1)*c*T
        if delta < 0:
            epsilon = 0
        else:
            b = (c-T+np.sqrt(delta))/(2*beta-2)
            if (b*(1-z)*initial_budget+1)*np.exp(-b*(1-z)*initial_budget) <= delta_g:
                epsilon = np.random.gamma(2, 1/b, 1)
            else:
                epsilon = 0
                      
    return epsilon
        
        

Overwriting schedule_utils.py


In [ ]:
from tfx.dsl.component.experimental.decorators import component
from tfx.v1.dsl.components import InputArtifact, OutputArtifact, Parameter, OutputDict
from tfx.types.standard_artifacts import Examples, Float
import numpy as np
import random
import shutil
from tfx.types import standard_component_specs
from tfx.utils import proto_utils
from tfx.components.example_gen.import_example_gen import executor
import schedule_utils

feature_num = 14

def sparse_to_dense(value):
    if isinstance(value, tf.sparse.SparseTensor):
        return tf.sparse.to_dense(value)
    return value

def wrap_int64(val):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=val))

def wrap_float32(val):
    return tf.train.Feature(float_list=tf.train.FloatList(value=val))


@component
def Scheduler(data: InputArtifact[Examples],
              significance: InputArtifact[String],
              blocks: OutputArtifact[Examples],
              data_root: Parameter[str],
              info_root: Parameter[str],
              schedule_type: Parameter[int],  # 0 for sage; 1 for infocom; 2 for rpbs; 3 for extended rpbs
              gamma: Parameter[float],    # gamma is the neighborhood error range
              lambd: Parameter[float]):   # lambd is the probability lower bound, presetted when the pipeline is created
    train_uri = os.path.join(data.uri, 'Split-train')
    train_filenames = [os.path.join(train_uri, name) for name in os.listdir(train_uri)]
    dataset = tf.data.TFRecordDataset(train_filenames, compression_type='GZIP')

    feature_description = {}
    feature_description['feature'] = tf.io.VarLenFeature(tf.float32)
    feature_description['label'] = tf.io.VarLenFeature(tf.float32)
    feature_description['block'] = tf.io.VarLenFeature(tf.int64)
    
    tags = ['feature', 'label', 'block']
    
    def read_and_decode(example_string):
        feature_dict = tf.io.parse_single_example(example_string, feature_description)
        outputs = [feature_dict[tags[i]] for i in range(len(tags))]
        return outputs
    
    records = dataset.map(read_and_decode)
    
    sig_list = []
    sig_uri = significance.uri
    f = open(sig_uri, 'r')
    for sig in f.readlines():
        sig_list.append(float(sig))
    f.close()
    print("Sig_list: ", sig_list)
    
    info_dir = os.path.join(info_root, 'budget.info')
    eps_dir = os.path.join(info_root, 'allocation.info')
    block_dir = os.path.join(data_root, 'filter/filter.tfrecord')
    
    sensitivity = 1.0 * feature_num
    
    info = np.loadtxt(info_dir) ## default delimiter is " "
    eps_array = np.array([[0.], [0.001]])
    block_list = list(range(1, (info.shape[1]+1)))
    impl_Flag = True
    eps0 = 1.0
    # Randomly choose a block
    while(impl_Flag):
        idx = random.choice(block_list)
        block_list.remove(idx)
        significance = sig_list[(idx-1)]
        print("Significacne: ", significance)
        initial_budget = info[0, idx-1]
        consumed_budget = info[1, idx-1]
        
        # Block Retire
        if consumed_budget > initial_budget:
            continue
        
        args = [initial_budget, consumed_budget, lambd, gamma, sensitivity, significance, eps0]
        
        # Allocation Strategy            
        epsilon = schedule_utils.schedule(schedule_type=schedule_type, arguments=args)
        
        if epsilon>0:
            info[1, idx-1] += (epsilon+eps0)
            eps_array = np.concatenate((eps_array, np.array([[idx], [epsilon]])), axis=1)
            print("Epislon: ", epsilon)
            
        # Stop Rule
        if eps_array.shape[1] > 5 or len(block_list)==0:
            impl_Flag = False
    
            
    # Update the info file
    np.savetxt(info_dir, info, fmt='%f')
    
    # Save the allocation result
    np.savetxt(eps_dir, eps_array, fmt='%f')
    print('Eps array: ', eps_array)
    
    # Package the selected blocks as an artifact
    def package(record):
        content = {}
        content['feature'] = wrap_float32(sparse_to_dense(record[0]).numpy())
        content['label'] = wrap_float32(sparse_to_dense(record[1]).numpy())
        content['block'] = wrap_int64(sparse_to_dense(record[2]).numpy())
        feature = tf.train.Features(feature=content)
        example = tf.train.Example(features=feature)
        serialized = example.SerializeToString()
        
        return serialized
    
    allocation_list = np.delete(eps_array[0, :], 0)
    print("Allocation: ", allocation_list)
    
    with tf.io.TFRecordWriter(block_dir) as writer:
        for record in records:
            idx = sparse_to_dense(record[-1])
            print("IDX: ", idx)
            if idx in allocation_list:
                print("idx: ", idx)
                writer.write(package(record))
    
    filter_input_cfg = Input(splits=[example_gen_pb2.Input.Split(name='train', pattern='filter/*'),
                                    example_gen_pb2.Input.Split(name='eval', pattern='eval/*')])
    
    filter_output_cfg = example_gen_pb2.Output()
    
    output_dict = {standard_component_specs.EXAMPLES_KEY: [blocks]}
    exec_properties = {standard_component_specs.INPUT_CONFIG_KEY: proto_utils.proto_to_json(filter_input_cfg),
                      standard_component_specs.OUTPUT_CONFIG_KEY: proto_utils.proto_to_json(filter_output_cfg),
                      standard_component_specs.INPUT_BASE_KEY: data_root,
                      standard_component_specs.OUTPUT_DATA_FORMAT_KEY: example_gen_pb2.FORMAT_TF_EXAMPLE}
    example_gen = executor.Executor()
    example_gen.Do({}, output_dict, exec_properties)

In [ ]:
scheduler = Scheduler(data=example_gen.outputs['examples'],
                      significance=estimator.outputs['significance'],
                      data_root='/home/liweiting/adult/data/',
                      info_root='/home/liweiting/adult/data/info',
                      schedule_type=2, gamma=10.0, lambd=0.95)
context.run(scheduler)

In [ ]:
statistics_gen = StatisticsGen(examples=scheduler.outputs['blocks'])
context.run(statistics_gen)

In [ ]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'], infer_feature_shape=False)
context.run(schema_gen)

In [ ]:
_adult_transform_module_file = 'adult_transform.py'

In [ ]:
%%writefile {_adult_transform_module_file}

import tensorflow as tf 
import tensorflow_transform as tft
import numpy as np
import sys
import noise_utils
import tensorflow_probability as tfp

noise_type = 0
feature_num = 14

def sparse_to_dense(value):
    if isinstance(value, tf.sparse.SparseTensor):
        return tf.sparse.to_dense(value)
    return value

def transformed_name(key):
    return key + '_xf'

def process_block(inputs):
    feature, epsilon, block = inputs
    tf.print("Blockkkkk: ", block)
    tf.print("Epsilonnn: ", epsilon)
    tf.print("Feature: ", feature)
    feature = tf.reshape(feature, [-1, feature_num])
    
    # Calculate the sensitivity of this task
    sensitivity = 1.0 * feature_num
    
    # add noise to the training data blocks
    is_eval = tf.math.equal(block, tf.constant(0, dtype=tf.int64))   
    def not_process(x):
        return x
    
    feature_processed = tf.cond(is_eval, lambda: not_process(feature), lambda: noise_utils.laplace_noise_input(feature, sensitivity, epsilon))

    return feature_processed
    

def preprocessing_fn(inputs, custom_config):
    outputs = {}
    
    # Read the epsilon allocation result
    eps_array = np.loadtxt(custom_config)
    block_list = tf.cast(tf.convert_to_tensor(eps_array[0,:]), tf.int64)
    eps_list = tf.convert_to_tensor(eps_array[1,:])
    
    # Whether this block in the chosen block list
    blocks = sparse_to_dense(inputs['block'])
    equal = tf.math.equal(blocks, block_list)
    
    def get_epsilon(eps_idx):
        idx = tf.where(eps_idx)
        epsilon = eps_list[idx[0][0]]
        return tf.cast(epsilon, tf.float32)
    
    epsilon = tf.vectorized_map(get_epsilon, equal)
    tf.print("Epsilon: ", epsilon)
    dense = sparse_to_dense(inputs['feature'])
    print("Dense: ", dense)
    
    if noise_type == 0:
        noised = tf.map_fn(process_block, (dense, epsilon, blocks), fn_output_signature=tf.float32)
        outputs[transformed_name('feature')] = tf.reshape(noised, [-1, feature_num])
    else:
        reshape = tf.reshape(dense, [-1, feature_num])
        outputs[transformed_name('feature')] = reshape
    
    label_dense = sparse_to_dense(inputs['label'])
    outputs[transformed_name('label')] = tf.reshape(label_dense, [-1,])
    
    def assign_epsilon(inputs):
        # not necessarily label, only need its shape
        label, epsilon = inputs
        epsilon_tensor = tf.ones_like(label)
        return epsilon*epsilon_tensor
    
    epsilons = tf.vectorized_map(assign_epsilon, (label_dense, epsilon))
    outputs['epsilon'] = tf.reshape(epsilons, [-1,])

    return outputs

In [ ]:
transform = Transform(examples=scheduler.outputs['blocks'],
                      schema=schema_gen.outputs['schema'],
                      module_file=os.path.abspath(_adult_transform_module_file),
                      custom_config='/home/liweiting/adult/data/info/allocation.info')
context.run(transform)

In [17]:
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]
print(tfrecord_filenames)
# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

for tfrecord in dataset.take(1):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)
#   print(sparse_to_dense(tfrecord))

c=0
for tfrecord in dataset:
    c+=1
print(c)

['/tmp/tfx-interactive-2021-11-04T14_17_50.622915-d2barbhg/ImportExampleGen/examples/1/Split-train/data_tfrecord-00000-of-00001.gz']
features {
  feature {
    key: "block"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "feature"
    value {
      float_list {
        value: 0.42465752363204956
        value: 0.3333333432674408
        value: 0.11691994220018387
        value: 0.6000000238418579
        value: 0.800000011920929
        value: 0.3333333432674408
        value: 0.23076923191547394
        value: 0.0
        value: 1.0
        value: 1.0
        value: 0.0
        value: 0.0
        value: 0.44897958636283875
        value: 0.949999988079071
        value: 0.6027397513389587
        value: 0.5
        value: 0.07210604846477509
        value: 0.6000000238418579
        value: 0.800000011920929
        value: 0.3333333432674408
        value: 0.8461538553237915
        value: 0.0
        value: 1.0
        value: 1.0
        value: 0

11


In [20]:
_noise_utils_file = 'noise_utils.py'

In [21]:
%%writefile {_noise_utils_file}

import numpy as np 
import tensorflow as tf 
import tensorflow_probability as tfp

def laplace_noise_input(raw, sensitivity, epsilon):
    laplace = tfp.distributions.Laplace(0.0, (sensitivity/epsilon))
    def add_noise(clean):
        noise = laplace.sample(sample_shape=clean.shape)
        tf.print("Noise shape: ", noise.shape)
        noised = tf.math.add(noise, clean)
        
        return noised
    
    noised = tf.vectorized_map(add_noise, raw)

    return noised

def laplace_noise_gradient(gradient, sensitivity, epsilon, epoch):
    clipped_gradients = tf.clip_by_value(gradient, clip_value_min=-(sensitivity/2), clip_value_max=(sensitivity/2))
    eps_iter = epsilon/epoch
    laplace = tfp.distributions.Laplace(0.0, (sensitivity/eps_iter))
    noise = laplace.sample(sample_shape=gradient.shape)
    noised = tf.math.add(noise, gradient)
  
    return noised

Overwriting noise_utils.py


In [13]:
_adult_model_file = 'adult_model.py'

In [9]:
%%writefile {_adult_model_file}
import tensorflow as tf 
from tensorflow.keras.layers import Dense, Softmax, BatchNormalization
from tensorflow.keras import Model
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import noise_utils

# _noise_type = 1
# _clip_value = 0.01
# _epoch = 40

class NN(Model):
    def __init__(self):
        super(NN, self).__init__()
        self.d1 = Dense(8, activation='sigmoid', name='fc1')
        self.bn = BatchNormalization()
        self.d2 = Dense(1, activation='sigmoid', name='fc2')

    def call(self, x):
        x = self.d1(x)
        x = self.bn(x)
        x = self.d2(x)
        return x
    
#     def add_noise(self, inputs):
#         gradient, epsilon = inputs
#         noised = tf.nest.map_structure(lambda g:noise_utils.laplace_noise_gradient(g, self.clip_value, epsilon, self.epoch), gradient)
        
#         return noised
        
#     def train_step(self, data):
#         # feature_dict has 'epsilon' and 'feature_xf'
#         feature_dict, labels = data
#         with tf.GradientTape() as tape:
#             predictions = self(feature_dict['feature_xf'], training=True)
#             loss = self.compiled_loss(labels, predictions)
#             regularization = sum(self.losses)
#             loss += regularization
  
#         if self.noise_type == 1:
#             jacobian = tape.jacobian(loss, self.trainable_variables)
#             gradient = tf.vectorized_map(self.add_noise, (jacobian, feature_dict['epsilon']))
#             gradient = tf.nest.map_structure(lambda g:tf.math.reduce_mean(g, axis=0), gradient)
#             self.optimizer.apply_gradients(zip(gradient, self.trainable_variables))
#         else:
#             gradient = tape.gradient(loss, self.trainable_variables)            
#             self.optimizer.apply_gradients(zip(gradient, self.trainable_variables))

#         self.compiled_metrics.update_state(labels, predictions)
#         return {m.name: m.result() for m in self.metrics}

#     def test_step(self, data):
#         feature_dict, labels = data
#         predictions = self(feature_dict['feature_xf'], training=False)
#         self.compiled_metrics.update_state(labels, predictions)

#         return {m.name: m.result() for m in self.metrics}

    
if __name__ == "__main__":
    # Initialized a random model 
    model = NN()
    inputs = tf.random.uniform((1, 14))
    outputs = model(inputs)
    model.summary()
    model.save_weights('saved_model/')
    print("Model Saved!")
    

Overwriting adult_model.py


In [13]:
_adult_trainer_module_file = 'adult_trainer.py'

In [15]:
%%writefile {_adult_trainer_module_file}

import tensorflow as tf 
import tensorflow_transform as tft

from tfx.components.trainer.fn_args_utils import FnArgs
from typing import List, Text
from tfx.components.trainer.fn_args_utils import DataAccessor
from tfx_bsl.tfxio import dataset_options
import noise_utils
from adult_model import NN
from tensorflow.keras.metrics import BinaryAccuracy, AUC, Precision

noise_type = 0
_epoch = 100
_clip_value = 0.01

class NNNoised(NN):
    def __init__(self, clip_value, epoch):
        super(NNNoised, self).__init__()
        self.clip_value = clip_value
        self.epoch = epoch

    def add_noise(self, inputs):
        gradient, epsilon = inputs
        noised = tf.nest.map_structure(lambda g:noise_utils.laplace_noise_gradient(g, self.clip_value, epsilon, self.epoch), gradient)

        return noised

    def train_step(self, data):
        # feature_dict has 'epsilon' and 'feature_xf'
        feature_dict, labels = data
        with tf.GradientTape() as tape:
            predictions = self(feature_dict['feature_xf'], training=True)
            loss = self.compiled_loss(labels, predictions)
            regularization = sum(self.losses)
            loss += regularization

        jacobian = tape.jacobian(loss, self.trainable_variables)
        gradient = tf.vectorized_map(self.add_noise, (jacobian, feature_dict['epsilon']))
        gradient = tf.nest.map_structure(lambda g:tf.math.reduce_mean(g, axis=0), gradient)
        self.optimizer.apply_gradients(zip(gradient, self.trainable_variables))

        self.compiled_metrics.update_state(labels, predictions)
        return {m.name: m.result() for m in self.metrics}
    
    def test_step(self, data):
        feature_dict, labels = data
        predictions = self(feature_dict['feature_xf'], training=False)
        self.compiled_metrics.update_state(labels, predictions)

        return {m.name: m.result() for m in self.metrics}


class NNUnnoised(NN):
    def __init__(self, epoch):
        super(NNUnnoised, self).__init__()
        self.epoch = epoch
        
    def train_step(self, data):
        feature_dict, labels = data
        with tf.GradientTape() as tape:
            predictions = self(feature_dict['feature_xf'], training=True)
            loss = self.compiled_loss(labels, predictions, regularization_losses=self.losses)
        
        gradient = tape.gradient(loss, self.trainable_variables)            
        self.optimizer.apply_gradients(zip(gradient, self.trainable_variables))
        self.compiled_metrics.update_state(labels, predictions)
        
        return {m.name: m.result() for m in self.metrics}
    
    def test_step(self, data):
        feature_dict, labels = data
        predictions = self(feature_dict['feature_xf'], training=False)
        self.compiled_metrics.update_state(labels, predictions)

        return {m.name: m.result() for m in self.metrics}

    
def build_keras_model(model_dir):
    if noise_type == 1:
        model = NNNoised(_clip_value, _epoch)
        loss_custom = tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
    else:
        model = NNUnnoised(_epoch)
        loss_custom = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    model.load_weights(model_dir)
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(0.01, 215, 0.99, staircase=True)
    optimizer = tf.keras.optimizers.Adam()
    model.compile(optimizer=optimizer, loss=loss_custom, metrics=[AUC()], run_eagerly=False)
    
    return model


def input_fn(file_pattern: List[Text],
             data_accessor: DataAccessor,
             tf_transform_output: tft.TFTransformOutput,
             batch_size: int = 200) -> tf.data.Dataset:
    return data_accessor.tf_dataset_factory(
            file_pattern,
            dataset_options.TensorFlowDatasetOptions(
              batch_size=batch_size, label_key='label_xf'),
            tf_transform_output.transformed_metadata.schema).repeat()


def _get_serve_tf_examples_fn(model, tf_transform_output):
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
#         feature_spec.pop('label')
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)
        print("Transformed_features: ", transformed_features)
        return model(transformed_features['feature_xf'])

    return serve_tf_examples_fn

def run_fn(fn_args: FnArgs):
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

    train_dataset = input_fn(fn_args.train_files, fn_args.data_accessor,
                                tf_transform_output, 250)
    eval_dataset = input_fn(fn_args.eval_files, fn_args.data_accessor,
                               tf_transform_output, 250)

    model = build_keras_model('/home/liweiting/adult/saved_model/')

    # Write logs to path
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=fn_args.model_run_dir, update_freq='batch')

    model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      epochs=model.epoch,
      verbose=2,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps,
      callbacks=[tensorboard_callback])

    signatures = {
      'serving_default':
          _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
                  tf.TensorSpec(shape=[None], dtype=tf.string, name='transformed_examples'))
    }
    
    print("Saved_path: ", fn_args.serving_model_dir)
#     model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Overwriting adult_trainer.py


In [14]:
trainer = Trainer(module_file=os.path.abspath(_adult_trainer_module_file),
                                examples=transform.outputs['transformed_examples'],
                                transform_graph=transform.outputs['transform_graph'],
                                schema=schema_gen.outputs['schema'],
                                train_args=TrainArgs(num_steps=30),
                                eval_args=EvalArgs(num_steps=16))
context.run(trainer)

INFO:absl:Generating ephemeral wheel package for '/home/liweiting/adult/adult_trainer.py' (including modules: ['write_tfrecord', 'write_info', 'adult_transform', 'noise_utils', 'adult_trainer', 'adult_model', 'schedule_utils', 'divide_block', 'exp2', 'exp1', 'exp3']).
INFO:absl:User module package has hash fingerprint version c43de51853b80251c64428b29b5914d022467aab88c027f6ebc9f209baaeb556.
INFO:absl:Executing: ['/home/liweiting/tfx_air/pipeline_36/bin/python', '/tmp/tmpsgxgd5ys/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmpzojlld7s', '--dist-dir', '/tmp/tmpyugw0fnf']
INFO:absl:Successfully built user code wheel distribution at '/tmp/tfx-interactive-2021-11-29T14_32_39.763171-949fpu06/_wheels/tfx_user_code_Trainer-0.0+c43de51853b80251c64428b29b5914d022467aab88c027f6ebc9f209baaeb556-py3-none-any.whl'; target user module is 'adult_trainer'.
INFO:absl:Full user module path is 'adult_trainer@/tmp/tfx-interactive-2021-11-29T14_32_39.763171-949fpu06/_wheels/tfx_user_code_T

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Assign: 2, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Assign: 2, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Assign: 2, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verb

INFO:absl:Feature epsilon has a shape . Setting to DenseTensor.
INFO:absl:Feature feature_xf has a shape dim {
  size: 14
}
. Setting to DenseTensor.
INFO:absl:Feature label_xf has a shape . Setting to DenseTensor.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Assign: 2, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Assign: 2, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


INFO:absl:Feature epsilon has a shape . Setting to DenseTensor.
INFO:absl:Feature feature_xf has a shape dim {
  size: 14
}
. Setting to DenseTensor.
INFO:absl:Feature label_xf has a shape . Setting to DenseTensor.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Assign: 2, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Assign: 2, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Assign: 2, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verb

INFO:absl:Feature epsilon has a shape . Setting to DenseTensor.
INFO:absl:Feature feature_xf has a shape dim {
  size: 14
}
. Setting to DenseTensor.
INFO:absl:Feature label_xf has a shape . Setting to DenseTensor.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Assign: 2, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Assign: 2, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Epoch 1/100
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing 

Epoch 59/100
30/30 - 0s - loss: 0.5803 - auc: 0.5963 - val_loss: 0.0000e+00 - val_auc: 0.7053
Epoch 60/100
30/30 - 2s - loss: 0.5815 - auc: 0.5956 - val_loss: 0.0000e+00 - val_auc: 0.7101
Epoch 61/100
30/30 - 1s - loss: 0.5825 - auc: 0.5973 - val_loss: 0.0000e+00 - val_auc: 0.7161
Epoch 62/100
30/30 - 1s - loss: 0.5790 - auc: 0.5955 - val_loss: 0.0000e+00 - val_auc: 0.7085
Epoch 63/100
30/30 - 1s - loss: 0.5817 - auc: 0.5938 - val_loss: 0.0000e+00 - val_auc: 0.7161
Epoch 64/100
30/30 - 0s - loss: 0.5794 - auc: 0.6004 - val_loss: 0.0000e+00 - val_auc: 0.7100
Epoch 65/100
30/30 - 0s - loss: 0.5812 - auc: 0.5936 - val_loss: 0.0000e+00 - val_auc: 0.7154
Epoch 66/100
30/30 - 0s - loss: 0.5794 - auc: 0.6030 - val_loss: 0.0000e+00 - val_auc: 0.7086
Epoch 67/100
30/30 - 0s - loss: 0.5787 - auc: 0.5997 - val_loss: 0.0000e+00 - val_auc: 0.7098
Epoch 68/100
30/30 - 0s - loss: 0.5815 - auc: 0.5980 - val_loss: 0.0000e+00 - val_auc: 0.7121
Epoch 69/100
30/30 - 1s - loss: 0.5788 - auc: 0.5954 - val_l

RuntimeError: run_fn failed to generate model.

In [19]:
eval_config = tfma.EvalConfig(
      model_specs=[tfma.ModelSpec(label_key='label')],
      slicing_specs=[tfma.SlicingSpec()],
      metrics_specs=[
          tfma.MetricsSpec(metrics=[
              tfma.MetricConfig(
                  class_name='BinaryAccuracy',
                  threshold=tfma.MetricThreshold(
                      value_threshold=tfma.GenericValueThreshold(
                          lower_bound={'value': 0.8})))
          ])
      ])

evaluator = Evaluator(examples=scheduler.outputs['blocks'], model=trainer.outputs['model'], eval_config=eval_config)
context.run(evaluator)

INFO:absl:Running driver for Evaluator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Evaluator
INFO:absl:Nonempty beam arg extra_packages already includes dependency
ERROR:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "metrics": [\n        {\n          "class_name": "BinaryAccuracy",\n          "threshold": {\n            "value_threshold": {\n              "lower_bound": 0.8\n            }\n          }\n        }\n      ]\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "label"\n    }\n  ],\n  "slicing_specs": [\n    {}\n  ]\n}', 'feature_slicing_spec': None, 'fairness_indicator_thresholds': None, 'example_splits': 'null', 'module_file': None, 'module_path': None} 'custom_eval_shared_model'
INFO:absl:Request was made to ignore the baseline ModelSpec and any change thresholds. This is likely because a baseline model was not provided: updated_config=
model_specs {
  label_key: "label"
}
slicing_specs {
}
m

Epsilon:  [0.001]
Blockkkkk:  [0]
Epsilonnn:  0.001
Feature:  [0.287671238 0.333333343 0.0516768806 ... 0 0.397959173 0.925]


INFO:absl:Evaluation complete. Results written to /tmp/tfx-interactive-2021-11-25T10_37_01.833177-8sj1z8xl/Evaluator/evaluation/9.
INFO:absl:Checking validation results.


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


INFO:absl:Blessing result True written to /tmp/tfx-interactive-2021-11-25T10_37_01.833177-8sj1z8xl/Evaluator/blessing/9.
INFO:absl:Running publisher for Evaluator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 14
        type_id: 20
        uri: "/tmp/tfx-interactive-2021-11-25T10_37_01.833177-8sj1z8xl/Evaluator/evaluation/9"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.2"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "ModelEvaluation"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 15
        type_id: 21
        uri: "/tmp/tfx-interactive-2021-11-25T10_37_01.833177-8sj1z8xl/Evaluator/blessing/9"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "current_model"
          value {
            string_value: "/tmp/tfx-interactive-2021-11-25T10_37_01.833177-8sj1z8xl/Trainer/model/8"
          }
        }
        custom_properties {
          key: "current_model_id"
          value {
            int_value: 12
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.2"
          }
        }
        state: LIVE
        , artifact_type: id: 21
        name: "ModelBlessing"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [20]:
output_path = evaluator.outputs['evaluation'].get()[0].uri
eval_result = tfma.load_eval_result(output_path)
validation_result = tfma.load_validation_result(output_path)
print("EVAL result: ", eval_result)
print("Validation result: ", validation_result)

EVAL result:  EvalResult(slicing_metrics=[((), {'': {'': {'binary_accuracy': {'doubleValue': 0.8383572697639465}}}})], plots=[((), None)], attributions=[((), None)], config=model_specs {
  label_key: "label"
}
slicing_specs {
}
metrics_specs {
  metrics {
    class_name: "BinaryAccuracy"
    threshold {
      value_threshold {
        lower_bound {
          value: 0.8
        }
      }
    }
  }
  model_names: ""
}
, data_location='<user provided PCollection>', file_format='<unknown>', model_location='/tmp/tfx-interactive-2021-11-25T10_37_01.833177-8sj1z8xl/Trainer/model/8/Format-Serving')
Validation result:  validation_ok: true
validation_details {
  slicing_details {
    slicing_spec {
    }
    num_matching_slices: 1
  }
}

